<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/AgeGenderModelLoadingFunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 1: Import Libraries and Set Up Environment
